In [14]:
import psycopg2
import os
import sys
import requests
import datetime
import time
import re

In [15]:
# db연결
conn = psycopg2.connect(host="ec2-23-21-207-93.compute-1.amazonaws.com", 
                        dbname="d3oubpekvnbupv", 
                        user="grxhirqndvyqvv", 
                        password="6f1afaafe16d245c70666bdb8c831aa876e62b380a1544f5dc832c51f27cece6", 
                        port="5432")

# 데이터 조작 객체 생성
cur = conn.cursor()

In [23]:
#데이터베이스 새로 만들기 (복합 키 세팅)
# 테이블명 : nesdata
# 컬럼명 : nesdate(뉴스작성일), newsname(뉴스타이틀), neslink(뉴스링크)
# create table
def db_create():
    cur.execute("CREATE TABLE IF NOT EXISTS newsdata(newsdate VARCHAR(30) NOT NULL, newsname VARCHAR(100) NOT NULL, newslink VARCHAR(200), primary key(newsname, newslink));")
    conn.commit()

db_create()

In [24]:
# db에 넣기 통합본 - 자동용
def insertnews():

    # db연결
    conn = psycopg2.connect(host="ec2-23-21-207-93.compute-1.amazonaws.com", 
                            dbname="d3oubpekvnbupv", 
                            user="grxhirqndvyqvv", 
                            password="6f1afaafe16d245c70666bdb8c831aa876e62b380a1544f5dc832c51f27cece6", 
                            port="5432")

    # 데이터 조작 객체 생성
    cur = conn.cursor()

    # 네이버 검색api 키값
    client_id = 'CmmN2OJp47c60h9IWTRs'
    client_secret = 'mALduPDQEB'

    #검색설정
    search_word = '지하철 사고'
    encode_type = 'json'
    max_display = 30
    sort = 'date'
    start = 1

    url = f"https://openapi.naver.com/v1/search/news.{encode_type}?query={search_word}&display={str(int(max_display))}&start={str(int(start))}&sort={sort}"

    #헤더에 아이디와 키 정보 넣기
    headers = {'X-Naver-Client-Id' : client_id,
            'X-Naver-Client-Secret':client_secret
            }

    #HTTP요청 보내기
    r = requests.get(url, headers=headers)
    #요청 결과 보기 200 이면 정상적으로 요청 완료

    # json형식으로 받기
    result = r.json()

    # key : items
    result_items=result['items']

    # db에 저장하기

    for i in result_items:
        
        # 날짜 형식 변환
        pDate = datetime.datetime.strptime(i['pubDate'][5:-6], '%d %b %Y %H:%M:%S')
        pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
        
        # 뉴스제목 디코딩
        title = clean_html(i['title'])
        
        link = i['link']
        
        insertdata = (pDate, title, link)
        
        # ON CONFLICT : 데이터가 중복되면 INSERT하지않음
        cur.execute("INSERT into newsdata(newsdate, newsname, newslink) VALUES (%s, %s, %s) ON CONFLICT (newsname, newslink) DO NOTHING", insertdata)

    # db commit
    conn.commit()   

In [25]:
insertnews()

In [26]:
# select newdata
cur.execute(f"SELECT * FROM newsdata order by newsdate DESC limit 10")
result_all = cur.fetchall()

newslist=""
for i in result_all:
    for j in i:
        newslist = newslist + j + "\n"
    newslist = newslist + "\n"

print(newslist[:-2])

2022-11-18 22:15:00
퇴근길 이어 출근길, 연이틀 멈춰선 신림선...시민만 불편
https://n.news.naver.com/mnews/article/052/0001815578?sid=102

2022-11-18 20:59:00
퇴근길 이어 출근길, 연이틀 멈춰선 신림선...시민만 불편
https://n.news.naver.com/mnews/article/052/0001815549?sid=102

2022-11-18 20:58:00
만석 또 만석, 출근길 ‘발동동’…“증차 먼저 해야”
https://n.news.naver.com/mnews/article/032/0003187245?sid=102

2022-11-18 20:36:00
인천사랑병원, 사랑의 김장 나눔 동참 外
http://www.newsmp.com/news/articleView.html?idxno=228164

2022-11-18 20:36:00
경기도 광역버스 오늘부터 ‘입석 금지’…이용객들은 ‘출·퇴근 걱정’
http://www.kukinews.com/newsView/kuk202211180149

2022-11-18 20:27:00
안전사고 우려 입석 중단 첫날 표정‥버스 늘려달라
https://n.news.naver.com/mnews/article/214/0001236190?sid=102

2022-11-18 20:26:00
어제 멈춘 신림선, 오늘 오전 또 운행중단…45분만에 재개
http://www.polinews.co.kr/news/article.html?no=517626

2022-11-18 20:17:00
[여적] 경전철 신림선 고장
https://n.news.naver.com/mnews/article/032/0003187231?sid=110

2022-11-18 18:40:00
퇴근길 이어 출근길, 연이틀 멈춰선 신림선...시민만 불편
https://n.news.naver.com/mnews/article/052/0001815523?sid=102

2022-11-18 1

In [3]:
conn.commit()